# Query Million Song Dataset

If you have not already done so, make sure to run `cache_msd.ipynb` to cache msd to `msd.pickle` before running this notebook.

## Imports

In [3]:
import sys
sys.path.append('../python')

import utils
import dill as pickle
import time

## Load MSD Cache

In [4]:
start_time = time.time()
with open('msd.pickle', 'r') as f:
    msd = pickle.load(f)
print('Loaded in {:.2f} seconds'.format(time.time() - start_time))

Loaded in 9.96 seconds


## Query dataset

Below are some examples of AND and OR million song dataset queries that can be made against the cached msd dataset.

In [5]:
## All songs by artists w/ terms 'jazz' AND a hotttnesss greater than 0.5
query = {
    'song_hotttnesss': lambda hotttnesss: hotttnesss > 0.6,
    'artist_terms': ['electronic'],
    'song_year': lambda year: year > 1990,
    'type': 'AND'
}

results = utils.query_dict_array(msd, query)

Once the query is complete, lets create a new directory and populate it with symlinks to the files pointed to by the results.

In [7]:
msd_path = '/home/bbpwn2/Documents/code/midi-dataset/data/lmd_matched/{}'
symlink_dir = 'symlink_test'

In [ ]:
tracks = [msd_path.format(r['path']) for r in results]
utils.create_symlink_dir(tracks, symlink_dir, file_limit_per_track=1)
print('symlinks created in directory: {}'.format(symlink_dir))